# Chapter 12: Quantum Encoding


---

**Prerequisites:**
- See `Chapter02_QuantumSoftware.ipynb` for installation instructions


In [ ]:
# Setup and imports
import numpy as np
import matplotlib.pyplot as plt
import math
import itertools
from scipy.sparse import diags
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.quantum_info import Statevector, Operator
from qiskit.circuit.library import UnitaryGate, QFTGate
from qiskit.circuit.library import QFT, phase_estimation, HamiltonianGate
from qiskit.quantum_info import SparsePauliOp
from Chapter03_EngineeringOptimization_functions import PlaneStressCantilever
from qiskit.circuit.library import  StatePreparation,DiagonalGate, ZGate, XGate
from Chapter03_EngineeringOptimization_functions import (truss2x2,truss3x2,truss2x3,truss3x3,truss_10bar,
                                                         PlaneStressCantilever)
from Chapter08_QuantumGates_functions import simulate_statevector, simulate_measurements #type: ignore
from Chapter12_QuantumEncoding_functions import (LCU_Ax,Pauli_Block_Encoding)

# Single Ry rotation

In [ ]:
phi = [np.sqrt(10)/5, np.sqrt(15)/5]
theta = 2 * np.arctan2(phi[1], phi[0])
qc = QuantumCircuit(1)
qc.ry(theta, 0)
qc.draw("mpl")
state = Statevector.from_instruction(qc)
display(state.draw("latex"))
display(qc.draw("mpl"))
print(f"theta: {theta:.4f} radians, {theta*180/np.pi:.2f} degrees")

## Mottonen decomposition

In [ ]:

# psi = [np.sqrt(0.3), np.sqrt(0.1), np.sqrt(0.2), np.sqrt(0.4)]

# Total prob for q1=0 is 0.4, for q1=1 is 0.6
theta_1 = 2 * np.arctan2(np.sqrt(0.6), np.sqrt(0.4))

# If q1=0, split 0.4 into 0.3 and 0.1
theta_00 = 2 * np.arctan2(np.sqrt(0.1), np.sqrt(0.3))

# If q1=1, split 0.6 into 0.2 and 0.4
theta_10 = 2 * np.arctan2(np.sqrt(0.4), np.sqrt(0.2))

qc = QuantumCircuit(2)
qc.ry(theta_1, 1)
qc.cry(theta_00, 1, 0, ctrl_state='0')
qc.cry(theta_10, 1, 0, ctrl_state='1') 


# results
print(theta_1,theta_00, theta_10)
state = Statevector.from_instruction(qc)
print("Resulting Statevector:")
display(state.draw('latex'))
display(qc.draw('mpl'))

## Initializing to Bell State

In [ ]:
theta_10 = 2*np.arctan2(1, 0)
qc = QuantumCircuit(2)
qc.h( 1)
qc.cry(theta_10, 1, 0, ctrl_state='1') 


# results
print(theta_1,theta_00, theta_10)
state = Statevector.from_instruction(qc)
print("Resulting Statevector:")
display(state.draw('latex'))
display(qc.draw('mpl'))

In [ ]:

p = [0.05, 0.10, 0.15, 0.10, 0.20, 0.15, 0.10, 0.15]

# Level 1
theta_2   = 2*np.arctan2(np.sqrt(0.60), np.sqrt(0.40))
# Level 2
theta_02  = 2*np.arctan2(np.sqrt(0.25), np.sqrt(0.15))
theta_12  = 2*np.arctan2(np.sqrt(0.25), np.sqrt(0.35))
# Level 3
theta_002 = 2*np.arctan2(np.sqrt(p[1]), np.sqrt(p[0]))
theta_012 = 2*np.arctan2(np.sqrt(p[3]), np.sqrt(p[2]))
theta_102 = 2*np.arctan2(np.sqrt(p[5]), np.sqrt(p[4]))
theta_112 = 2*np.arctan2(np.sqrt(p[7]), np.sqrt(p[6]))

qc = QuantumCircuit(3)
# Level 1: rotate q2
qc.ry(theta_2, 2)
# Level 2: rotate q1 conditioned on q2
qc.cry(theta_02, 2, 1, ctrl_state='0')
qc.cry(theta_12, 2, 1, ctrl_state='1')
# Level 3: rotate q0 conditioned on (q2, q1)
qc.ccx(2, 1, 0)  # placeholder: see note below
qc.cry(theta_002, 2, 0, ctrl_state='0')  
qc.cry(theta_012, 2, 0, ctrl_state='1') 

state = Statevector.from_instruction(qc)
print("Resulting Statevector:")
display(state.draw('latex'))
display(qc.draw('mpl'))

## Qiskit Initialize

In [ ]:
x = np.array([np.sqrt(2), 1, 0,-1])
n = int(np.ceil(np.log2(len(x))))
# Normalize the vector 
x = x / np.linalg.norm(x)

# Create circuit and prepare state
qc = QuantumCircuit(n)
qc.initialize(x, range(n))
display(qc.draw('mpl'))

## PREP-SELECT-UNPREP A*x for 2x2 matrix 

In [ ]:
# 1. Define A and |x>
# A = 2Z - 0.1X
A = np.array([
    [2, -0.1],
    [-0.1, -2]
], dtype=float)

x = np.array([1, 0], dtype=complex)  # |0> state
expected_output = A @ x

# 2. Hardcoded decomposition: A = 2*Z + (-0.1)*X
c0, c1 = 2.0, -0.1
alpha = np.abs(c0) + np.abs(c1)  # 2.1

num_ancilla = 1
num_system = 1

qr_sys = QuantumRegister(num_system, 'sys')
qr_anc = QuantumRegister(num_ancilla, 'anc')
qc = QuantumCircuit(qr_sys, qr_anc)  # System on top, ancilla on bottom

# 3. PREP: Encode sqrt(|c0|/alpha), sqrt(|c1|/alpha)
prep_vec = np.array([np.sqrt(np.abs(c0)/alpha), np.sqrt(np.abs(c1)/alpha)])
prep_gate = StatePreparation(prep_vec, label='Prep')
qc.append(prep_gate, qr_anc)


# 4. SELECT: Apply controlled unitaries

# Term 0: Controlled-Z when ancilla = |0⟩, coefficient c0 = 2.0
phase0 = np.angle(c0)  # 0
gate_circ_0 = QuantumCircuit(num_system, global_phase=phase0)
gate_circ_0.append(ZGate(), [0])
ctrl_z = gate_circ_0.to_gate(label='Z').control(
    num_ctrl_qubits=num_ancilla,
    ctrl_state='0'
)
qc.append(ctrl_z, [qr_anc[0], qr_sys[0]])


# Term 1: Controlled-X when ancilla = |1⟩, coefficient c1 = -0.1
phase1 = np.angle(c1)  # π
gate_circ_1 = QuantumCircuit(num_system, global_phase=phase1)
gate_circ_1.append(XGate(), [0])
ctrl_x = gate_circ_1.to_gate(label='X(π)').control(
    num_ctrl_qubits=num_ancilla,
    ctrl_state='1'
)
qc.append(ctrl_x, [qr_anc[0], qr_sys[0]])

# 5. UNPREP
qc.append(prep_gate.inverse(), qr_anc)

# 6. Simulate and post-select
full_init = Statevector.from_label('0' * num_ancilla).expand(Statevector(x))
final_sv = full_init.evolve(qc)

# Post-selection: ancilla = |0⟩
stride = 2**num_ancilla

res_vector = final_sv.data[0:2**num_system] * alpha
print("="*50)
print("LCU Result: ", np.round(np.real_if_close(res_vector), 4))
print("Classical:  ", np.round(np.real_if_close(expected_output), 4))
print("="*50)

qc.draw('mpl')

## Pauli Expansion

In [ ]:
A = np.array([
    [1, 0, 0, 0.5],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0.5, 0, 0, 1]
])
pauliSplit = SparsePauliOp.from_operator(A)
print(pauliSplit.paulis)
print(pauliSplit.coeffs)

## Performing Ax using LCU_Ax

In [ ]:
A = np.array([
    [1, 0, 0, 0.5],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0.5, 0, 0, 1]
], dtype=float)

x = np.array([0.6, 0.8, 0, 0], dtype=complex)

expected_output = A @ x
# Create circuit
qc, metadata = LCU_Ax(A, x)
print(metadata)
# Get the exact statevector
final_state = Statevector(qc)
num_system = metadata['num_system']
alpha = metadata['alpha']

res_vector = final_state.data[:2**num_system] * alpha
success_prob = np.linalg.norm(final_state.data[:2**num_system])**2

print("Expected output:", np.round(np.real_if_close(expected_output), 4))
print("Quantum output:", np.round(np.real_if_close(res_vector), 4))
print("Success probability:", np.round(success_prob, 4))
qc.draw('mpl')

## Choose from Examples

In [ ]:
example = 6
if (example == 1):
	A = np.array([[1,0],[0,1]]) 
	b = np.array([1,0])
elif (example == 2):
	A = np.array([[2,-1],[-1,2]])
	b = np.array([1,1])/np.sqrt(2)
elif (example == 3):
	A = np.array([[1,0,0,-0.5],[0,1,0,0],[0,0,1,0],[-0.5,0,0,1]])
	b = np.array([1,0,0,0])
elif (example == 4):
	A = np.array([[1.5,0.5],[0.5,1.5]])
	b = np.array([1,0])
elif (example == 5):
	p = 1
	A = np.array([[5*(10**p),-1],[-1,5]])
	b = np.array([1,0])
elif (example == 6):
	N = 2**5
	values = [-np.ones(N-1),2*np.ones(N),-np.ones(N-1)]
	A = diags(values,[-1,0,1]).toarray()/4
	b = np.zeros(N)
	b[0] = 1

print("A shape:", A.shape)
print("b shape:", b.shape)

## Block encoding

In [ ]:
U_matrix, metadata = Pauli_Block_Encoding(A, mode='statevector')
alpha = metadata['alpha']
print("U shape:", U_matrix.shape)
print("alpha:", alpha)

## LCU encoding of truss stiffness matrices

In [ ]:
fem_model= truss3x3()
fem_model.plot_truss()
K_full = fem_model.assemble_stiffness()
print("Size of K_full:", K_full.shape)
all_dofs = set(range(fem_model.n_dofs))
free_dofs = list(all_dofs - set(fem_model.fixed_dofs))
K = K_full[np.ix_(free_dofs, free_dofs)]
f = fem_model.loads[free_dofs]
print("Size of K:", K.shape)
N = 8
A = K[:N, :N]  # Just take a 4x4 block for testing
print("Size of sub-matrix A:", A.shape)
pauliSplit = SparsePauliOp.from_operator(A)
print("Number of Pauli terms:",pauliSplit.coeffs.shape[0])

# Resource requirements
num_ancilla = int(np.ceil(np.log2(pauliSplit.coeffs.shape[0])))  # One ancilla per Pauli term
num_system = int(np.ceil(np.log2(N)))
print(f"Ancilla qubits needed: {num_ancilla}")
print(f"System qubits needed: {num_system}")
print(f"Total qubits: {num_ancilla + num_system}")

# Normalization constant
alpha = np.sum(np.abs(pauliSplit.coeffs))
print(f"Normalization α: {alpha:.4e}")

## FEM Matrix Circuit analysis

In [ ]:
# Create a simple test vector
x = np.zeros(8)
x[0] = 1.0  # Unit displacement at first DOF

# Create LCU circuit
qc, metadata = LCU_Ax(A, x, mode='statevector')

# Analyze circuit
print(f"Circuit depth: {qc.depth()}")
print(f"Number of qubits: {qc.num_qubits}")

# Decompose to basis gates
qc_decomposed = qc.decompose(reps=3)
print(f"\nAfter decomposition to basis gates:")
print(f"Circuit depth: {qc_decomposed.depth()}")
print(f"Total gates: {sum(qc_decomposed.count_ops().values())}")

## Block encoding